# Importando e tratando os dados

In [23]:
!pip install kagglehub

In [24]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("asaniczka/tmdb-movies-dataset-2023-930k-movies")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/tmdb-movies-dataset-2023-930k-movies


In [25]:
import pandas as pd
colunas_escolhidas = ["id", "title", "vote_average", "vote_count", "release_date", "revenue", "runtime", "adult", "budget", "overview", "popularity", "genres", "keywords"]
dados = pd.read_csv(f"{path}/TMDB_movie_dataset_v11.csv", usecols=colunas_escolhidas)

In [26]:
print(f" Os seus dados estão com a seguinte configuração: {dados.shape}\n")
dados.head(2)

 Os seus dados estão com a seguinte configuração: (1249710, 13)



,id,title,vote_average,vote_count,release_date,revenue,runtime,adult,budget,overview,popularity,genres,keywords
0,27205,Inception,8.364,34495,2010-07-15,825532764,148,False,160000000,"Cobb, a skilled thief who commits corporate es...",83.952,"Action, Science Fiction, Adventure","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,2014-11-05,701729206,169,False,165000000,The adventures of a group of explorers who mak...,140.241,"Adventure, Drama, Science Fiction","rescue, future, spacecraft, race against time,..."


In [27]:
colunas = list(dados.columns)
for coluna in colunas:
  print(coluna)

id
title
vote_average
vote_count
release_date
revenue
runtime
adult
budget
overview
popularity
genres
keywords


In [28]:
dados['keywords'].value_counts()

,count
keywords,
short film,10641
woman director,8357
gay pornography,8088
stand-up comedy,2588
concert,1982
...,...
evil doctor,1
"media, earth, mother earth, planet earth",1
"chicago, illinois, sexual obsession, lover, delusion, mother-in-law, seduction, illusion, man of one's dreams, cabin in the woods, love affair, heiress, socialite, seductress, sexual relationship, personal diary, childlike, hopeless romantic, romance novel obsession, engaged man",1


In [29]:
dados.isnull().sum()

,0
id,0
title,13
vote_average,0
vote_count,0
release_date,237080
revenue,0
runtime,0
adult,0
budget,0
overview,268744


#CLEIVER - Versão 2 KNN

In [30]:
!pip install kagglehub

In [31]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("asaniczka/tmdb-movies-dataset-2023-930k-movies")
colunas_escolhidas = ["id", "title", "vote_average", "vote_count", "release_date", "revenue", "runtime", "adult", "budget", "overview", "popularity", "genres", "keywords"]
dados = pd.read_csv(f"{path}/TMDB_movie_dataset_v11.csv", usecols=colunas_escolhidas)

In [32]:
dados_cleiver = dados['genres']
dados_cleiver.fillna('Desconhecido', inplace=True)
display(dados_cleiver.head(2))

,genres
0,"Action, Science Fiction, Adventure"
1,"Adventure, Drama, Science Fiction"


In [33]:
#OneHot Encoding dos gêneros
genres_dummies = dados_cleiver.str.get_dummies(sep=', ')

In [34]:
#Exportar como csv para o Igor
genres_dummies.to_csv("one_hot_label_igor.csv", index=False)

#Acabou aqui

Aqui temos a tabela inteira com os gêneros corrigidos pelo one-hot encoding

In [35]:
genres_dummies = pd.read_csv("one_hot_label_igor.csv")

In [36]:
#Concatenar gêneros com one_hot e tabela original, para sabermos quais filmes são referenciados
#dados = dados.drop('genres', axis=1)

df_final = pd.concat([dados, genres_dummies], axis=1)
display(df_final.head(2))
print(df_final.shape)

,id,title,vote_average,vote_count,release_date,revenue,runtime,adult,budget,overview,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,27205,Inception,8.364,34495,2010-07-15,825532764,148,False,160000000,"Cobb, a skilled thief who commits corporate es...",...,0,0,0,0,0,1,0,0,0,0
1,157336,Interstellar,8.417,32571,2014-11-05,701729206,169,False,165000000,The adventures of a group of explorers who mak...,...,0,0,0,0,0,1,0,0,0,0


(1249710, 33)


Extrair ano da data de lançamento

In [37]:
df_final['release_date'] = pd.to_datetime(df_final['release_date'])
df_final['release_year'] = df_final['release_date'].dt.year
df_final = df_final.drop('release_date', axis=1)
df_final = df_final.dropna(subset=['title'])

#Inputation das datas faltantes


In [45]:
media_ano = df_final['release_year'].mean()
df_final['release_year'] = df_final['release_year'].fillna(value=media_ano)

Vamos aplicar o MinMaxScaler para normalizar os dados numéricos para nossso KNN

In [46]:
from sklearn.preprocessing import MinMaxScaler

#Aproveitando para arrumar booleano da coluna Adult
df_final['adult'] = df_final['adult'].astype(int)

# 1. Inicialize o scaler
scaler = MinMaxScaler()

# 2. Defina as colunas que você quer escalar
colunas_para_escalar = ['vote_average', 'vote_count', 'release_year', 'revenue', 'runtime', 'budget', 'popularity']

# 3. Use fit_transform nas colunas selecionadas
df_final[colunas_para_escalar] = scaler.fit_transform(df_final[colunas_para_escalar])

print("\nDataFrame com Variáveis Escaladas:")
display(df_final.head(2))


DataFrame com Variáveis Escaladas:


,id,title,vote_average,vote_count,revenue,runtime,adult,budget,overview,popularity,...,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western,release_year
0,27205,Inception,0.8364,1.000000,0.165107,0.012199,0,0.160,"Cobb, a skilled thief who commits corporate es...",0.028037,...,0,0,0,0,1,0,0,0,0,0.702341
1,157336,Interstellar,0.8417,0.944224,0.140346,0.013654,0,0.165,The adventures of a group of explorers who mak...,0.046835,...,0,0,0,0,1,0,0,0,0,0.715719


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
import pandas as pd

# Supondo que seu DataFrame se chama 'df'

# --- 1. Seleção de Features ---
# Vamos selecionar as colunas numéricas que já tratamos e consideramos úteis
colunas_numericas = ['vote_average', 'runtime', 'popularity', 'release_year', 'adult']
df_numerico = df_final.drop(['keywords', 'overview', 'title'], axis=1)

# Criamos um novo DataFrame apenas com elas, garantindo que não há NaNs
# (Assumindo que já fizemos a imputação da mediana ou removemos as linhas com nulos)
####df_numerico = df_final[colunas_numericas].dropna()

print("--- DataFrame com Features Numéricas ---")
print(df_numerico.head())

#Verificar se não temos nulos que atrapalharão o modelo
print(df_numerico.isnull().sum())

Treinamento do modelo KNN

In [ ]:
# --- 2. Escalonamento ---
# Todos os dados precisam estar na mesma escala para o KNN funcionar bem
scaler = MinMaxScaler()
features_escalonadas = scaler.fit_transform(df_numerico)

print("\n--- Amostra das Features Escalonadas (Array NumPy) ---")
print(features_escalonadas[:5])

# --- 3. Treinamento do Modelo ---
# n_neighbors=6 porque o primeiro vizinho (distância 0) será o próprio filme
# algorithm='brute' é direto, mas para datasets maiores 'kd_tree' ou 'ball_tree' podem ser mais rápidos
knn_model = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='euclidean')

# "Treinamos" o modelo com nossos dados escalonados
knn_model.fit(features_escalonadas)

print("\nModelo KNN treinado com sucesso!")

In [62]:
import pandas as pd
# Certifique-se de que numpy também está importado se ainda não estiver
import numpy as np

# --- Mapeamento de Título para Índice (seu código, sem alterações) ---
# Usamos o índice do df_numerico para selecionar apenas os títulos correspondentes no df original
# A linha abaixo pode gerar um índice com títulos duplicados, mas nossa função irá lidar com isso.
indices = pd.Series(df_numerico.index, index=dados.loc[df_numerico.index]['title'])


# --- A Função de Recomendação (VERSÃO CORRIGIDA E ROBUSTA) ---

def recomendar_filmes(titulo, modelo_knn, dados_escalonados, mapeamento_indices):
    """
    Recomenda 5 filmes similares a um filme de entrada com base em features numéricas.
    Versão robusta que lida com títulos duplicados.
    """
    # 1. Obter o índice (ou índices) do filme a partir do título
    try:
        idx_lookup = mapeamento_indices[titulo]

        # --- A CORREÇÃO ESTÁ AQUI ---
        # 2. Verificamos se a busca retornou múltiplos índices (uma pd.Series)
        #    Se sim, pegamos apenas o primeiro índice da lista.
        if isinstance(idx_lookup, pd.Series):
            idx = idx_lookup.iloc[0]
        else:
            # Se for apenas um número, usamos ele mesmo
            idx = idx_lookup

    except KeyError:
        return f"Filme '{titulo}' não encontrado no nosso banco de dados. Tente outro."

    # 3. Obter as distâncias e os índices dos 5 vizinhos mais próximos
    distancias, indices_vizinhos = modelo_knn.kneighbors([dados_escalonados[idx]])

    # 4. Os índices retornados são para o array 'dados_escalonados'.
    # O primeiro vizinho (índice 0) é o próprio filme, então pulamos ele
    indices_filmes_similares = indices_vizinhos[0][1:]

    # 5. Traduzir os índices de volta para títulos de filmes
    titulos_recomendados = dados['title'].iloc[indices_filmes_similares].tolist()

    return titulos_recomendados

# --- Exemplo de Uso (seu código, com uma pequena melhoria para imprimir a mensagem de erro) ---
# Vamos pegar um filme famoso para testar
filme_exemplo = 'Iron Man' # Ou qualquer outro filme que estava dando erro

recomendacoes = recomendar_filmes(filme_exemplo, knn_model, features_escalonadas, indices)

print(f"Porque você assistiu '{filme_exemplo}', talvez você goste de:")

# Verifica se a função retornou uma lista (sucesso) ou uma string (erro)
if isinstance(recomendacoes, list):
    for i, filme in enumerate(recomendacoes):
        print(f"{i+1}. {filme}")
else:
    # Imprime a mensagem de erro, como "Filme não encontrado..."
    print(recomendacoes)

Porque você assistiu 'Iron Man', talvez você goste de:
1. Guardians of the Galaxy
2. Mad Max: Fury Road
3. Captain America: The First Avenger
4. Iron Man 2
5. Iron Man 3


In [ ]:
#Recomendar Interstellar, Iron Man - ficou bom

In [56]:
# Use um título de filme que você sabe que causou o erro
titulo_com_erro = 'Interstellar'

# Vamos inspecionar o que nosso mapa de índices retorna
resultado_da_busca = indices[titulo_com_erro]

print(f"Resultado da busca por '{titulo_com_erro}':")
print(resultado_da_busca)
print(f"\nTipo do resultado: {type(resultado_da_busca)}")

Resultado da busca por 'Interstellar':
1

Tipo do resultado: <class 'numpy.int64'>
